# Imports

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime

# State Data

In [2]:
res = requests.get('https://cdn-api.co-vin.in/api/v2/admin/location/states')
df_states = pd.DataFrame(json.loads(res.text)["states"])
df_states.head()

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [3]:
df_states.shape

(37, 2)

In [4]:
df_states.to_csv('./Data/State_Names.csv', index = False)

# Disctricts Data

In [5]:
unique_state_ids = df_states['state_id'].unique().tolist()

In [6]:
df_districts = None
for state_id in unique_state_ids:
    res = requests.get(f'https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state_id}')
    temp_df = pd.DataFrame(json.loads(res.text)['districts'])
    temp_df['state_id'] = state_id
    if df_districts is None:
        df_districts = temp_df
    else:
        df_districts = pd.concat([df_districts, temp_df], ignore_index=True)

In [7]:
df_districts.shape

(756, 3)

In [8]:
df_districts.to_csv('./Data/Disctrict_Names.csv', index = False)

# Combine State And Districts

In [9]:
df_full = pd.merge(df_districts, df_states, on='state_id')
df_full.head()

,district_id,district_name,state_id,state_name
0,3,Nicobar,1,Andaman and Nicobar Islands
1,1,North and Middle Andaman,1,Andaman and Nicobar Islands
2,2,South Andaman,1,Andaman and Nicobar Islands
3,9,Anantapur,2,Andhra Pradesh
4,10,Chittoor,2,Andhra Pradesh


In [10]:
df_full.to_csv('./Data/State_And_Disctrict_Names.csv', index = False)

# Get Vaccine Availability

In [11]:
DIST_ID = '446'
START_DATE = datetime.today().strftime('%d-%m-%Y')

In [12]:
VAX_DATA_PORTAL = f'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={DIST_ID}&date={START_DATE}'

In [13]:
res = requests.get(VAX_DATA_PORTAL)
vax_info_df = pd.DataFrame(json.loads(res.text)['centers'])
vax_info_df.head()

,center_id,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions
0,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,[{'session_id': '2456762c-fd53-4e6f-baeb-a722f...
1,555866,Capital Hospital BBSR,Unit 6 Capital Hospital Bhubaneswar,Odisha,Khurda,Bhubaneswar MC,751001,20,85,15:00:00,18:00:00,Free,[{'session_id': 'b831f7ed-1d74-41a3-ac66-0528c...
2,624523,Health Village Hospital BBSR,15 1 Civic Centre IRC Village Nayapalli Bhuban...,Odisha,Khurda,Bhubaneswar MC,751015,20,85,09:00:00,18:00:00,Paid,[{'session_id': 'd7cf310d-2413-4263-aa4f-07e26...
3,577315,UPHC POKHARIPUT,Antanta Vihar Plot No 149 Pokhariput Bhubaneswar,Odisha,Khurda,Bhubaneswar MC,751006,20,85,15:00:00,18:00:00,Free,[{'session_id': 'a861c1ae-2822-4d9a-aa24-92179...
4,503376,JANLA HOSPITAL CVC,JANLA HOSPITAL,Odisha,Khurda,Jatani,752054,20,85,09:00:00,18:00:00,Free,[{'session_id': '45c6a95e-b42f-491e-9750-de9b3...


In [14]:
vax_info_all_sessions_df = vax_info_df.explode('sessions').reset_index(drop=True)
vax_info_all_sessions_df['Min_Age'] = vax_info_all_sessions_df.sessions.apply(lambda x: x['min_age_limit'])
vax_info_all_sessions_df['Doses_Available'] = vax_info_all_sessions_df.sessions.apply(lambda x: x['available_capacity'])
vax_info_all_sessions_df['Vaccine'] = vax_info_all_sessions_df.sessions.apply(lambda x: x['vaccine']).fillna('UNKNOWN')
vax_info_all_sessions_df['Date'] = vax_info_all_sessions_df.sessions.apply(lambda x: x['date'])

vax_info_all_sessions_df.head()

,center_id,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,Min_Age,Doses_Available,Vaccine,Date
0,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,{'session_id': '2456762c-fd53-4e6f-baeb-a722fe...,18,0,COVAXIN,03-05-2021
1,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,{'session_id': 'e7ff8b8e-61e5-4030-9e14-3c143f...,18,0,COVAXIN,04-05-2021
2,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,{'session_id': '3d0277b4-8eac-4a4e-a5de-7cdde3...,18,0,COVAXIN,05-05-2021
3,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,{'session_id': 'efce6cc7-8dad-4058-b5e3-e462f7...,18,0,COVAXIN,06-05-2021
4,693349,Baramunda UPHC WN50 (Age18-44),Near Nilakanteswar Mandir Sapatabdhi Nagar Bhu...,Odisha,Khurda,Bhubaneswar MC,751003,20,85,08:30:00,13:00:00,Free,{'session_id': '9b1c3e74-50aa-4eb8-8b70-04780a...,18,0,COVAXIN,07-05-2021


**As I have demonstrated we can pull these relevant data from web. Now let's build an app around it!**